In [14]:
# import necessary package
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime
from scipy.signal import savgol_filter
from sklearn.metrics import r2_score, mean_squared_error
import copy
import os
import datetime

In [15]:
# import Long Short Term Memory, Convolutional Neural Network and Random Forest model implementation
from ensemble_lstm import AutoLSTM
from ensemble_cnn import AutoCNN
from ensemble_rf import AutoRF

In [16]:
# suppress warnings
import warnings
warnings.filterwarnings('ignore')

## Read Data 

In [83]:
govbudget =  pd.read_excel('GovBudget.xls', parse_dates = True )
gs10 = pd.read_excel('GS10.xls',parse_dates = True )
m2 = pd.read_excel('M2REAL.xls',parse_dates = True )
diffusion = pd.read_excel('bos_difx.xls',parse_dates = True )
sp500_eps_index = pd.read_excel('S&P500_EPS_index.xls',parse_dates = True )

In [84]:
dfs = [diffusion,govbudget,gs10,m2]
d = sp500_eps_index

In [85]:
for df in dfs: 
    d = pd.merge(d, df, on =
                'Date')

In [86]:
d = d.rename(columns = {'SP500_EPS_index':"SP500-EPS-Index",'GAC': 'ActivityIndex', 'NOC': 'NewOrders', 'SHC': 'Shipments', 'UOC': 'UnfilledOrders',                                                                                                                                                 
                            'DTC': 'DeliveryTime', 'IVC': 'Inventories', 'PPC': 'PricesPaid', 'NEC': 'Employment', 'AWC': 'AverageWorkweek', 'M2REAL': 'M2', 'GovBudget': 'GovernmentBudgetBalance'}) 
d.head(1)

,Date,SP500-EPS-Index,ActivityIndex,NewOrders,Shipments,UnfilledOrders,DeliveryTime,Inventories,PricesPaid,PRC,...,DTF,IVF,PPF,PRF,NEF,AWF,CEF,GovernmentBudgetBalance,GS10,M2
0,1969-08-01,100.0,8.0,16.9,31.1,0.1,-5.4,3.9,38.2,5.7,...,-6.9,-1.7,59.2,39.0,11.6,-5.5,16.0,-5.624,6.69,1572.1


In [87]:
data = d.drop(['DTF', 'PRC', 'IVF', 'PPF', 'PRF', 'NEF', 'AWF', 'CEF', 'GAF', 'NOF', 'SHF', 'UOF'],axis = 1)

In [88]:
data.to_excel('data.xlsx', index = False)

In [89]:
data.head()

,Date,SP500-EPS-Index,ActivityIndex,NewOrders,Shipments,UnfilledOrders,DeliveryTime,Inventories,PricesPaid,Employment,AverageWorkweek,GovernmentBudgetBalance,GS10,M2
0,1969-08-01,100.000000,8.0,16.9,31.1,0.1,-5.4,3.9,38.2,2.0,2.6,-5.624,6.69,1572.1
1,1969-09-01,100.287869,-1.6,6.2,6.9,-4.1,-3.6,6.5,41.2,7.1,3.4,-5.624,7.16,1569.0
2,1969-10-01,99.682710,12.1,3.0,11.6,-20.1,-5.8,-2.5,35.3,3.8,4.8,-5.624,7.10,1564.1
3,1969-11-01,99.067314,-3.4,7.5,5.0,-5.7,-0.9,3.9,35.9,5.6,-2.0,-3.466,7.14,1561.1
4,1969-12-01,98.459021,-7.9,9.6,23.2,-15.5,-7.8,-5.0,37.3,2.3,1.4,-3.466,7.65,1559.4


In [90]:
def load_data(feature_files = ['bos_difx.xls', 'GS10.xls', 'M2REAL.xls', 'GovBudget.xls'], target = 'S&P500-EPS.xlsx', output = 'data.xlsx'):
        curr_path = os.getcwd()
        df_list = []
    # Load feature data from various resources from 1969/8/1 to the most recent date
        for file in feature_files:
            input_path = os.path.join(curr_path,file)
            df = pd.read_excel(
            input_path,
        )
            if file == 'GovBudget.xls':
                df['Date'] = pd.to_datetime(df['Date']).dt.to_period('M')       
                df = df.set_index('Date').resample('M').bfill()
                df = df.reset_index()
                df['Date'] = df['Date'].dt.strftime('%Y-%m-01')
                df['Date'] = pd.to_datetime(df['Date'])
        #df['Date'] = df['Date'].dt.strftime('%Y-%m')
                df_list.append(df)
            return df_list

In [91]:
data = pd.merge(sp500_eps_index, diffusion, on = 'Date')
data.head(1)

,Date,SP500_EPS_index,GAC,NOC,SHC,UOC,DTC,IVC,PPC,PRC,...,NOF,SHF,UOF,DTF,IVF,PPF,PRF,NEF,AWF,CEF
0,1969-08-01,100.0,8.0,16.9,31.1,0.1,-5.4,3.9,38.2,5.7,...,10.4,15.6,-6.8,-6.9,-1.7,59.2,39.0,11.6,-5.5,16.0


In [92]:
df 

,Date,M2REAL
0,1969-08-01,1572.1
1,1969-09-01,1569.0
2,1969-10-01,1564.1
3,1969-11-01,1561.1
4,1969-12-01,1559.4
...,...,...
623,2021-07-01,7558.4
624,2021-08-01,7630.6
625,2021-09-01,7658.5
626,2021-10-01,7656.5


In [34]:
df.head()

,Date,GovBudget
0,1969-08-01,-5.624
1,1969-09-01,-5.624
2,1969-10-01,-5.624
3,1969-11-01,-3.466
4,1969-12-01,-3.466


In [35]:
df.dtypes

Date         datetime64[ns]
GovBudget           float64
dtype: object